<a href="https://colab.research.google.com/github/Nine91011/Assignment-III-Data-Extraction-using-LLMs/blob/main/Asignment_III_Chayanon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install google-generativeai pdfplumber pandas openpyxl --quiet

In [ ]:
#!git clone https://github.com/Nine91011/Assignment-III-Data-Extraction-using-LLMs.git

Cloning into 'Assignment-III-Data-Extraction-using-LLMs'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (3/3), done.
Receiving objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)


In [1]:
import google.generativeai as genai
from google.colab import userdata
import pandas as pd
import json
import os
import re

In [2]:
os.chdir('/content/Assignment-III-Data-Extraction-using-LLMs')
print("Current working directory:", os.getcwd())

Current working directory: /content/Assignment-III-Data-Extraction-using-LLMs


In [ ]:
#!mkdir articles prompts output

In [3]:
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

In [42]:
# Load Full Text
pdf_file_path = "articles/15.pdf"
article_text = genai.upload_file(path=pdf_file_path)

In [43]:
article_text

genai.File({
    'name': 'files/37cr7acs6rfy',
    'display_name': '15.pdf',
    'mime_type': 'application/pdf',
    'sha256_hash': 'NTAxYzE1NDhlYjE1YjM5YmU5YjZhODk1MjA2NzExOTE0NDZmMWViMjdiODAwNTg1YjIzNGM4NTVjNWFiMTgyZQ==',
    'size_bytes': '1896686',
    'state': 'ACTIVE',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/37cr7acs6rfy',
    'create_time': '2025-04-30T15:43:57.717427Z',
    'expiration_time': '2025-05-02T15:43:57.680796866Z',
    'update_time': '2025-04-30T15:43:57.717427Z'})

In [44]:
### Initial Prompt ###

Extract_data_prompt = f"""

Please extract the following metadata from the full text of this research article and present it following Data Extraction Template.

For each variable:
– (Y) =  Allow multiple input
– (N) =  Not Allow multiple input

––––––––––––––––––––––––––––––––––––––––––––––
GENERAL STUDY INFORMATION
1. study_id (N)
   • Definition: Unique identifier for this study (e.g., "1")
2. author (N)
   • Definition: List of authors as in byline (e.g., "Justin A Strauss, Chun R Chao, …")
3. year (N)
   • Definition: Publication year (e.g., 2013)
4. title (N)
   • Definition: Full article title
5. journal (N)
   • Definition: Journal name (e.g., "J Am Med Inform Assoc")

STUDY DESIGN
6. study_design (N)
   • Definition: Overall design (e.g., "Validation study", "Retrospective cohort")
7. Study-phase (Y)
   • Definition: Clinical trial phase or stage (e.g., “Phase I”, “Phase II”)
8. Study period (Y)
   • Definition: Dates or years during which study ran (e.g., “2000–2007”)
9. study_task (N)
   • Definition: Main task performed (e.g., “Identify and extract cancer recurrence”)

DATA SOURCE
10. Data source (Y)
    • Definition: Origin of text data (e.g., “Electronic pathology reports”)
11. Data source type (Y)
    • Definition: Format of source (e.g., “SAS database”, “PDF”)
12. N of data sources (N)
    • Definition: Number of distinct sources (e.g., “1”)
13. Type of clinical text data (Y)
    • Definition: Kind of document (e.g., “Pathology report”)
14. Text language (Y)
    • Definition: Language of text (e.g., “English”)
15. Type of cancer (Y)
    • Definition: Cancer types studied (e.g., “breast”, “prostate”)
16. Recurrent organ (Y)
    • Definition: Organs where recurrence occurred (e.g., “breast”, “lymph node”)
17. Type of outcome (N)
    • Definition: Measured outcomes (e.g., “primary diagnosis”, “recurrence”)

DATA PREPARATION
18. vocab size (N)
    • Definition: Vocabulary size if text representation used (e.g., “–” for rule-based)
19. Text representation (Y)
    • Definition: Representation techniques (e.g., “BERT”, “TF-IDF”)

STUDY SUBJECT
20. total_n_patients (Y)
    • Definition: Total number of patients (e.g., “800”)
21. total_n_documents (Y)
    • Definition: Total number of documents processed (e.g., “882”)
22. Data split ratio (Y)
    • Definition: Training/validation/test split (e.g., “60:20:20”)
23. n_patients for each set (Y)
    • Definition: Number of patients per split (e.g., “480, 160, 160”)
24. n_documents for each set (Y)
    • Definition: Number of docs per split (e.g., “529, 176, 177”)

MODELING TECHNIQUES
25. Type of models (Y)
    • Definition: Model category (e.g., “Rule-based”, “ML”)
26. type of algorithm (Y)
    • Definition: Specific algorithm (e.g., “hierarchical rules”, “SVM”)
27. model-name (Y)
    • Definition: Model or tool name (e.g., “SCENT”)
28. Validation method (Y)
    • Definition: How performance was validated (e.g., “Manual chart review”)

EVALUATION METRICS
29. AUPRC (Y)
30. AUROC (Y)
31. F1 (Y)
32. Recall (Y)
33. Precision (Y)
34. Brier’s score (Y)
   • Definitions: Standard definitions for each metric

**Handling Multiple Analysis Units:**
Crucially, if the article analyzes or reports results for different subgroups, datasets, study phases, models/algorithms, or other conditions separately (especially involving variables marked 'Y' for multiple inputs like 'Type of cancer', 'Study-phase', 'model-name', 'Type of models', 'type of algorithm'), **you MUST create a separate JSON object for each distinct analysis unit or unique combination**.

Carefully examine the article to identify *all distinct groups or conditions* for which results are reported separately. Consider all combinations of key variables like 'Type of cancer', 'model-name', 'Type of models', 'Study-phase', 'Text representation', etc., that necessitate separate reporting.

Ensure each JSON object contains key-value pairs for *all* the variables listed in the template. Repeat common information (like author, year, title, journal) within each JSON object as necessary. If a specific variable (e.g., a particular evaluation metric like AUPRC) is not reported for a specific analysis unit, use `null` or an empty string "" for its value in that specific JSON object. Do not use lists like ['null'].

**Output Format Requirement:**
* Provide the output strictly as a **JSON list**.
* **Each element** in the list MUST be a **separate JSON object** representing **ONE unique study instance or analysis subgroup** identified according to the "Handling Multiple Analysis Units" section above.
* The keys within each JSON object MUST exactly match the variable names listed in the template (e.g., "study_id", "author", "Type of cancer", "model-name", "Recall").


Article Text:
---
{article_text}
---

Extracted Metadata:

"""

In [45]:
model = genai.GenerativeModel('models/gemini-2.5-pro-exp-03-25')

response = model.generate_content([Extract_data_prompt, article_text])

response_text = response.text
print(response_text)

```json
[
  {
    "study_id": "1",
    "author": "Karen E. Batch, Jianwei Yue, Alex Darcovich, Kaelan Lupton, Corinne C. Liu, David P. Woodlock, Mohammad Ali K. El Amine, Pamela I. Causa-Andrieu, Lior Gazit, Gary H. Nguyen, Farhana Zulkernine, Richard K. G. Do, Amber L. Simpson",
    "year": 2022,
    "title": "Developing a Cancer Digital Twin: Supervised Metastases Detection From Consecutive Structured Radiology Reports",
    "journal": "Frontiers in Artificial Intelligence",
    "study_design": "Validation study",
    "Study-phase": null,
    "Study period": [
      "July 1, 2009 - March 26, 2021"
    ],
    "study_task": "Supervised Metastases Detection",
    "Data source": [
      "Memorial Sloan Kettering Cancer Center"
    ],
    "Data source type": [
      "database"
    ],
    "N of data sources": "1",
    "Type of clinical text data": [
      "Structured radiology reports (CT)"
    ],
    "Text language": [
      "English"
    ],
    "Type of cancer": [
      "Metastatic disea

In [46]:
# --- Transform Data for Excel ---

# 1. Attempt to find and extract the JSON list string (still potentially brittle)
json_start_index = response_text.find('[')
json_end_index = response_text.rfind(']') + 1
json_output_string = response_text[json_start_index : json_end_index]

# 2. Parse the JSON string into a Python list (expected: list of dictionaries)
parsed_data = json.loads(json_output_string)

# 3. Directly create Pandas DataFrame from the list of dictionaries
#    Pandas automatically uses dictionary keys as columns and list elements as rows.
df = pd.DataFrame(parsed_data)
print("\nDataFrame Created (Row-based):")
print(df.to_string())

# --- Define Output Directory and Filename ---
output_directory = "output"
excel_filename = "extracted_data from '15'.xlsx" # Changed filename slightly
output_dir_path = os.path.join(os.getcwd(), output_directory)
os.makedirs(output_dir_path, exist_ok=True)
output_file_path = os.path.join(output_dir_path, excel_filename)

# --- Save DataFrame to Excel ---
df.to_excel(output_file_path, index=False, engine='openpyxl')
print(f"\nSAVE Excel successful: '{output_file_path}'")


DataFrame Created (Row-based):
   study_id                                                                                                                                                                                                                                author  year                                                                                                            title                               journal      study_design Study-phase                     Study period                       study_task                               Data source Data source type N of data sources           Type of clinical text data Text language        Type of cancer  Recurrent organ              Type of outcome vocab size Text representation total_n_patients total_n_documents                    Data split ratio n_patients for each set n_documents for each set       Type of models                                                    type of algorithm                          model-name 